In [64]:
import numpy as np
import matplotlib
from matplotlib import colors
import matplotlib.pyplot as plt
import geone
import geone.covModel as gcm
import geone.imgplot3d as imgplt3
import pyvista as pv
import sys
import os
import flopy as fp
import pyemu

sys.path.append("../../../")
#my modules
from ArchPy.base import *
from ArchPy.tpgs import *

In [65]:
tmp_model_ws = "reference"
sim = fp.mf6.MFSimulation.load(sim_ws=tmp_model_ws)
m = sim.get_model("test")

loading simulation...
  loading simulation name file...
  loading tdis package...
  loading model gwf6...
    loading package dis...
    loading package oc...
    loading package ic...
    loading package npf...
    loading package chd...
    loading package obs...
  loading solution package test...


In [66]:
sr = pyemu.helpers.SpatialReference.from_namfile(
        os.path.join(tmp_model_ws, "test.nam"),
        delr=m.dis.delr.array, delc=m.dis.delc.array)
sr
template_ws = "template"
pf = pyemu.utils.PstFrom(original_d=tmp_model_ws, new_d=template_ws,
                 remove_existing=True,
                 longnames=True, spatial_reference=sr,
                 zero_based=False,start_datetime="1-1-2025")

   could not remove start_datetime
2025-09-05 20:15:14.792400 starting: opening PstFrom.log for logging
2025-09-05 20:15:14.792400 starting PstFrom process
2025-09-05 20:15:14.795409 starting: setting up dirs
2025-09-05 20:15:14.795409 starting: removing existing new_d 'template'


Exception: unable to remove existing dir: template
[WinError 32] Le processus ne peut pas accéder au fichier car ce fichier est utilisé par un autre processus: 'template\\test.npf'

In [ ]:
tmp_model_ws

'reference'

## Observations

In [ ]:
df = pd.read_csv(os.path.join(tmp_model_ws,"head_obs_ref.csv"),index_col=0)
df

,H1,H2,H3,H4,H5,H6,H7,H8,H9,H10
time,,,,,,,,,,
1.0,0.73881,0.425,0.42667,0.11038,0.025822,0.037492,0.17995,0.048933,0.79004,0.26967


In [ ]:
# open(os.path.join(tmp_model_ws,"head_obs_ref.csv"))

In [ ]:
hds_df = pf.add_observations("head_obs_ref.csv",insfile="head_obs_ref.csv.ins", index_cols="time",
                    use_cols=list(df.columns.values),prefix="hds",)
hds_df

2025-09-05 13:58:03.833377 starting: adding observations from output file head_obs_ref.csv
2025-09-05 13:58:03.834380 starting: adding observations from tabular output file '['head_obs_ref.csv']'
2025-09-05 13:58:03.836377 starting: reading list-style file: template\head_obs_ref.csv
2025-09-05 13:58:03.845381 finished: reading list-style file: template\head_obs_ref.csv took: 0:00:00.009004
2025-09-05 13:58:03.849376 starting: building insfile for tabular output file head_obs_ref.csv
2025-09-05 13:58:03.866381 finished: building insfile for tabular output file head_obs_ref.csv took: 0:00:00.017005
2025-09-05 13:58:03.868377 starting: adding observation from instruction file 'template\head_obs_ref.csv.ins'
2025-09-05 13:58:03.886378 finished: adding observation from instruction file 'template\head_obs_ref.csv.ins' took: 0:00:00.018001
2025-09-05 13:58:03.888381 finished: adding observations from tabular output file '['head_obs_ref.csv']' took: 0:00:00.054001
2025-09-05 13:58:03.889379 fi

,obsnme,obsval,weight,obgnme
oname:hds_otype:lst_usecol:h10_time:1,oname:hds_otype:lst_usecol:h10_time:1,0.269670,1.0,oname:hds_otype:lst_usecol:h10
oname:hds_otype:lst_usecol:h1_time:1,oname:hds_otype:lst_usecol:h1_time:1,0.738810,1.0,oname:hds_otype:lst_usecol:h1
oname:hds_otype:lst_usecol:h2_time:1,oname:hds_otype:lst_usecol:h2_time:1,0.425000,1.0,oname:hds_otype:lst_usecol:h2
oname:hds_otype:lst_usecol:h3_time:1,oname:hds_otype:lst_usecol:h3_time:1,0.426670,1.0,oname:hds_otype:lst_usecol:h3
oname:hds_otype:lst_usecol:h4_time:1,oname:hds_otype:lst_usecol:h4_time:1,0.110380,1.0,oname:hds_otype:lst_usecol:h4
oname:hds_otype:lst_usecol:h5_time:1,oname:hds_otype:lst_usecol:h5_time:1,0.025822,1.0,oname:hds_otype:lst_usecol:h5
oname:hds_otype:lst_usecol:h6_time:1,oname:hds_otype:lst_usecol:h6_time:1,0.037492,1.0,oname:hds_otype:lst_usecol:h6
oname:hds_otype:lst_usecol:h7_time:1,oname:hds_otype:lst_usecol:h7_time:1,0.179950,1.0,oname:hds_otype:lst_usecol:h7
oname:hds_otype:lst_usecol:h8_time:1,oname:hds_otype:lst_usecol:h8_time:1,0.048933,1.0,oname:hds_otype:lst_usecol:h8
oname:hds_otype:lst_usecol:h9_time:1,oname:hds_otype:lst_usecol:h9_time:1,0.790040,1.0,oname:hds_otype:lst_usecol:h9


In [ ]:
[f for f in os.listdir(template_ws) if f.endswith(".ins")]

['head_obs_ref.csv.ins']

## Parameters

parameters are the surface elevation of unit C, stored as a txt file

In [ ]:
pf

In [ ]:
arr_file = "mean_C.txt"
tag = arr_file.split('.')[0]
pf.add_parameters(filenames=arr_file, par_type="grid",
                   par_name_base=tag, pargp=tag, zone_array=None,
                   par_style="multiplier",
                   upper_bound=10., lower_bound=0.1,
                   ult_ubound=10, ult_lbound=0.1)

2025-09-05 13:58:21.566432 transform was not passed, setting default tranform to 'log'
2025-09-05 13:58:21.566432 starting: adding grid type m style parameters for file(s) ['mean_C.txt']
2025-09-05 13:58:21.567430 starting: loading array template\mean_C.txt
2025-09-05 13:58:21.612007 finished: loading array template\mean_C.txt took: 0:00:00.044577
2025-09-05 13:58:21.613007 loaded array 'reference\mean_C.txt' of shape (70, 140)
2025-09-05 13:58:21.640297 starting: writing array-style template file 'template\mean_c_inst0_grid.csv.tpl'
2025-09-05 13:58:21.640297 starting: writing template file template\mean_c_inst0_grid.csv.tpl for ['mean_c_inst:0']
2025-09-05 13:58:21.974369 finished: writing template file template\mean_c_inst0_grid.csv.tpl for ['mean_c_inst:0'] took: 0:00:00.334072
2025-09-05 13:58:21.975369 starting: writing array-based template file 'template\mean_c_inst0_grid.csv.tpl'
2025-09-05 13:58:22.006899 finished: adding grid type m style parameters for file(s) ['mean_C.txt']

,parnme,parval1,i,j,x,y,pargp,tpl_filename,input_filename,partype,partrans,parubnd,parlbnd,scale,offset,dercom,parchglim
pname:mean_c_inst:0_ptype:gr_pstyle:m_i:0_j:0_x:0.75_y:104.25,pname:mean_c_inst:0_ptype:gr_pstyle:m_i:0_j:0_x:0.75_y:104.25,1.0,0,0,0.75,104.25,mean_c,template\mean_c_inst0_grid.csv.tpl,template\mult\mean_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,1,factor
pname:mean_c_inst:0_ptype:gr_pstyle:m_i:0_j:1_x:2.25_y:104.25,pname:mean_c_inst:0_ptype:gr_pstyle:m_i:0_j:1_x:2.25_y:104.25,1.0,0,1,2.25,104.25,mean_c,template\mean_c_inst0_grid.csv.tpl,template\mult\mean_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,1,factor
pname:mean_c_inst:0_ptype:gr_pstyle:m_i:0_j:2_x:3.75_y:104.25,pname:mean_c_inst:0_ptype:gr_pstyle:m_i:0_j:2_x:3.75_y:104.25,1.0,0,2,3.75,104.25,mean_c,template\mean_c_inst0_grid.csv.tpl,template\mult\mean_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,1,factor
pname:mean_c_inst:0_ptype:gr_pstyle:m_i:0_j:3_x:5.25_y:104.25,pname:mean_c_inst:0_ptype:gr_pstyle:m_i:0_j:3_x:5.25_y:104.25,1.0,0,3,5.25,104.25,mean_c,template\mean_c_inst0_grid.csv.tpl,template\mult\mean_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,1,factor
pname:mean_c_inst:0_ptype:gr_pstyle:m_i:0_j:4_x:6.75_y:104.25,pname:mean_c_inst:0_ptype:gr_pstyle:m_i:0_j:4_x:6.75_y:104.25,1.0,0,4,6.75,104.25,mean_c,template\mean_c_inst0_grid.csv.tpl,template\mult\mean_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,1,factor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pname:mean_c_inst:0_ptype:gr_pstyle:m_i:69_j:135_x:203.25_y:0.75,pname:mean_c_inst:0_ptype:gr_pstyle:m_i:69_j:135_x:203.25_y:0.75,1.0,69,135,203.25,0.75,mean_c,template\mean_c_inst0_grid.csv.tpl,template\mult\mean_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,1,factor
pname:mean_c_inst:0_ptype:gr_pstyle:m_i:69_j:136_x:204.75_y:0.75,pname:mean_c_inst:0_ptype:gr_pstyle:m_i:69_j:136_x:204.75_y:0.75,1.0,69,136,204.75,0.75,mean_c,template\mean_c_inst0_grid.csv.tpl,template\mult\mean_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,1,factor
pname:mean_c_inst:0_ptype:gr_pstyle:m_i:69_j:137_x:206.25_y:0.75,pname:mean_c_inst:0_ptype:gr_pstyle:m_i:69_j:137_x:206.25_y:0.75,1.0,69,137,206.25,0.75,mean_c,template\mean_c_inst0_grid.csv.tpl,template\mult\mean_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,1,factor
pname:mean_c_inst:0_ptype:gr_pstyle:m_i:69_j:138_x:207.75_y:0.75,pname:mean_c_inst:0_ptype:gr_pstyle:m_i:69_j:138_x:207.75_y:0.75,1.0,69,138,207.75,0.75,mean_c,template\mean_c_inst0_grid.csv.tpl,template\mult\mean_c_inst0_grid.csv,grid,log,10.0,0.1,1.0,0.0,1,factor


In [67]:
from shutil import copyfile
import fileinput

In [ ]:
def npf_tpl(k_layers, tmp_ws):
    pass
    # for riv in k_layers:
    #     riv_file = os.path.join(tmp_ws, riv)
    #     tpl_file = riv_file+".tpl"
    #     copyfile(riv_file, tpl_file)
        
    #     with fileinput.FileInput(tpl_file, inplace=True) as file:
    #         for line in file:
    #             print(line.replace("    0.00500000   ", "~    c_{}     ~".format(riv.split(".")[0])), end='')
    #     with open(tpl_file, 'r+') as f:
    #         content = f.read()
    #         line = "ptf ~"
    #         f.seek(0, 0)
    #         f.write(line.rstrip('\r\n') + "\n" + content)
        
    #     #add parameter to control file
    #     dic = {"parnme":"c_{}".format(riv.split(".")[0]),
    #           "parval1": 0.005,
    #            "pargp": "cond",
    #            "tpl_filename": tpl_file,
    #            'input_filename':riv_file,
    #            "partype" : "constant",
    #            "partrans" : "log",
    #            "parubnd" : 0.1,
    #            "parlbnd" : 0.0001,
    #            "parchglim" : "factor",
    #            "scale" : 1.0,
    #            "dercom" : 1,
    #            "offset" : 0.0} 
    #     a = pd.DataFrame(dic,index=["c_{}".format(riv.split(".")[0])])
    #     pf.par_dfs[0] = pf.par_dfs[0].append(a,sort=False)
        
    #     pf.tpl_filenames.append(riv+".tpl")
    #     pf.input_filenames.append(riv)

In [63]:
# with open("template/test.npf") as f:
#     print(f)

In [ ]:
arr_file = "test.npf"
tag = "k33"

2025-09-05 13:59:21.391227 transform was not passed, setting default tranform to 'log'
2025-09-05 13:59:21.393320 starting: adding constant type m style parameters for file(s) ['test.npf']
2025-09-05 13:59:21.393320 starting: loading array template\test.npf


ValueError: could not convert string 'BEGIN' to float64 at row 0, column 1.

## Command line

It will be necessary to add instruction to update the modflow model using ArchPy. First new surfaces are loaded into an ArchPy model created based on ArchPy_workspace. Then, archpy2modflow is created, new grid info are retrieved and applied with flopy to modflow model. In addition we have to modify BCs and obs to match new grid...
 inally, modflow 6 can be run.

In [22]:
# pf.pre_py_cmds.insert(0,"import sys")
pf.pre_py_cmds.append("import sys")
pf.pre_py_cmds.append("sys.path.append('..')")
pf.pre_py_cmds.append("from ArchPy_update import update_model")
# pf.pre_py_cmds.insert(1,"sys.path.append(os.path.join('..','..','..','pypestutils'))")

# ARCHPY # --> execute ArchPy_update.py script
pf.mod_py_cmds.append("update_model()")  # add archpy update script

# run modflow
pf.mod_sys_cmds.append("mf6")  # add modflow command
pst = pf.build_pst()

noptmax:0, npar_adj:9800, nnz_obs:10
2025-09-05 09:13:10.043959 forward_run line:pyemu.os_utils.run(r'mf6')



In [23]:
par = pst.parameter_data

In [41]:
par

,parnme,partrans,parchglim,parval1,parlbnd,parubnd,pargp,scale,offset,dercom,i,inst,j,pname,pstyle,ptype,x,y
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:0_x:0.75_y:104.25,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:0_x:0.75_y:104.25,log,factor,1.0,0.1,10.0,surf_c,1.0,0.0,1,0,0,0,surf,m,gr,0.75,104.25
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:1_x:2.25_y:104.25,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:1_x:2.25_y:104.25,log,factor,1.0,0.1,10.0,surf_c,1.0,0.0,1,0,0,1,surf,m,gr,2.25,104.25
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:2_x:3.75_y:104.25,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:2_x:3.75_y:104.25,log,factor,1.0,0.1,10.0,surf_c,1.0,0.0,1,0,0,2,surf,m,gr,3.75,104.25
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:3_x:5.25_y:104.25,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:3_x:5.25_y:104.25,log,factor,1.0,0.1,10.0,surf_c,1.0,0.0,1,0,0,3,surf,m,gr,5.25,104.25
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:4_x:6.75_y:104.25,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:0_j:4_x:6.75_y:104.25,log,factor,1.0,0.1,10.0,surf_c,1.0,0.0,1,0,0,4,surf,m,gr,6.75,104.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:135_x:203.25_y:0.75,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:135_x:203.25_y:0.75,log,factor,1.0,0.1,10.0,surf_c,1.0,0.0,1,69,0,135,surf,m,gr,203.25,0.75
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:136_x:204.75_y:0.75,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:136_x:204.75_y:0.75,log,factor,1.0,0.1,10.0,surf_c,1.0,0.0,1,69,0,136,surf,m,gr,204.75,0.75
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:137_x:206.25_y:0.75,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:137_x:206.25_y:0.75,log,factor,1.0,0.1,10.0,surf_c,1.0,0.0,1,69,0,137,surf,m,gr,206.25,0.75
pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:138_x:207.75_y:0.75,pname:surf_c_inst:0_ptype:gr_pstyle:m_i:69_j:138_x:207.75_y:0.75,log,factor,1.0,0.1,10.0,surf_c,1.0,0.0,1,69,0,138,surf,m,gr,207.75,0.75


In [ ]:
T1 = ArchPy.inputs.import_project("P1", "ArchPy_workspace")

### IMPORTING PROJECT P1 IN ArchPy_workspace DIRECTORY ### 

Surface_1_dic_surf_mean.npy
Unit A: Surface added for interpolation
Facies basement added to unit A ✅
Unit B: Surface added for interpolation
Facies Sand added to unit B ✅
Facies Gravel added to unit B ✅
Facies GM added to unit B ✅
Facies SM added to unit B ✅
Unit C: Surface added for interpolation
Facies Clay added to unit C ✅
Facies Silt added to unit C ✅
The SubPile object is not an Arch_table object
Unit D: Surface added for interpolation
Facies Sand added to unit D ✅
Facies SM added to unit D ✅
Unit D1: Surface added for interpolation
Facies Sand added to unit D1 ✅
Unit D2: Surface added for interpolation
Facies SM added to unit D2 ✅
Stratigraphic unit D added ✅
Stratigraphic unit C added ✅
Stratigraphic unit B added ✅
Stratigraphic unit A added ✅
Stratigraphic unit D2 added ✅
Stratigraphic unit D1 added ✅
Pile sets as Pile master
Property K added
## Adding Grid ##
## Grid added and is now simulation grid ##


 ##LOADING

In [13]:
T1.verbose=0

In [14]:
# T1.compute_surf(1)

In [15]:
# np.savetxt("template/surf_C.txt", T1.get_surfaces_unit(T1.get_unit("C"))[0])

In [16]:
# load new surfaces
surf_C = np.loadtxt("template/surf_C.txt")

In [17]:
# T1.get_pile_master().list_units

In [18]:
surfaces_by_piles = {}
surfaces_by_piles["P1"] = T1.Geol.surfaces_by_piles["P1"].copy()
surfaces_by_piles["PD"] = T1.Geol.surfaces_by_piles["PD"].copy()
surfaces_by_piles["P1"][0, 1] = surf_C  # update surface C

In [19]:
T1.define_domains(surfaces_by_piles)

In [20]:
import flopy as fp
import ArchPy.ap_mf
from ArchPy.ap_mf import archpy2modflow, array2cellids

In [21]:
archpy_flow = archpy2modflow(T1, exe_name="../../../../../exe/mf6.exe", model_dir="template")  # create the modflow model
archpy_flow.create_sim(grid_mode="layers", iu=0)  # create the simulation object and choose a certain discretization

Simulation created with the following parameters:
Grid mode: layers
To retrieve the simulation, use the get_sim() method


In [ ]:
sim = archpy_flow.get_sim()
gwf = archpy_flow.get_gwf()

In [30]:
m.dis.botm.set_data(gwf.dis.botm.array)

In [40]:
gwf.dis

package_name = dis
filename = test.dis
package_type = dis
model_or_simulation_package = model
model_name = test

Block options
--------------------
xorigin
{internal}
(0.0)

yorigin
{internal}
(0.0)

angrot
{internal}
(0.0)


Block dimensions
--------------------
nlay
{internal}
(5)

nrow
{internal}
(70)

ncol
{internal}
(140)


Block griddata
--------------------
delr
{constant 1.5}

delc
{constant 1.5}

top
{internal}
(array([[-6., -6., -6., ..., -6., -6., -6.],
       [-6., -6., -6., ..., -6., -6., -6.],
       [-6., -6., -6., ..., -6., -6., -6.],
       ...,
       [-6., -6., -6., ..., -6., -6., -6.],
       [-6., -6., -6., ..., -6., -6., -6.],
       [-6., -6., -6., ..., -6., -6., -6.]]))

botm
Layer_1{internal}
(array([[-6.55199337, -6.4551239 , -6.34888744, ..., -6.31776667,
        -6.47247314, -6.63935518],
       [-6.65084934, -6.5784502 , -6.47312832, ..., -6.28882074,
        -6.44636059, -6.58829975],
       [-6.71820974, -6.64620447, -6.55492592, ..., -6.3261075 ,
       